# Projeto: Classificação de Diabetes com Azure AutoML

Este notebook demonstra o processo de ponta-a-ponta para treinar um modelo de classificação usando o **Azure Machine Learning SDK v2** e o **Automated ML (AutoML)**.

O objetivo é prever a probabilidade de um paciente ter diabetes com base nos seus dados de saúde.

**Processo:**
1.  Conectar ao Workspace do Azure ML.
2.  Criar e Registrar o Ativo de Dados (Data Asset).
3.  Configurar e submeter o job de Automated ML.
4.  Analisar os resultados no Azure ML Studio.

## 1. Configuração do Ambiente

Primeiro, vamos importar as bibliotecas necessárias e verificar a versão do SDK do Azure ML.

In [ ]:
# Verificar a versão do SDK
!pip show azure-ai-ml

## 2. Conectar ao Workspace

Agora, vamos nos conectar ao nosso Workspace do Azure ML. Usamos o `DefaultAzureCredential` para autenticar e, em seguida, o `MLClient` para gerenciar os recursos do workspace.

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Verifica se a credencial padrão funciona
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Se falhar, usa a autenticação interativa (abre um browser)
    credential = InteractiveBrowserCredential()

In [ ]:
# Carrega os detalhes do workspace a partir do config.json
ml_client = MLClient.from_config(credential=credential)

# Imprime o nome do workspace para confirmar a conexão
print(f"Conectado ao workspace: {ml_client.workspace_name}")

## 3. Criar e Registrar o Ativo de Dados (Data Asset)

Esta é uma etapa crucial. O notebook original do tutorial assume que o *data asset* (ativo de dados) já existe, o que causa o erro `path can't be empty`.

Vamos corrigir isso: vamos pegar os arquivos de dados locais (a pasta `diabetes-data` que está neste mesmo diretório do notebook), carregá-los para o *datastore* padrão do Azure e registrá-los como um **Ativo de Dados** chamado `diabetes-training`.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# 1. Define o caminho local para os dados
# O './' significa 'nesta pasta atual'
local_data_path = "./diabetes-data/"

# 2. Define o Data Asset
# Damos a ele o nome 'diabetes-training', que o resto do notebook espera
my_data = Data(
    path=local_data_path,
    type=AssetTypes.MLTABLE,
    description="Diabetes training data MLTable for AutoML",
    name="diabetes-training"  # <-- Este é o nome que o job procura
)

# 3. Cria (ou atualiza) o Data Asset.
# Isto faz o UPLOAD dos arquivos locais para o Azure
ml_client.data.create_or_update(my_data)

print(f"Ativo de dados 'diabetes-training' criado com sucesso a partir do caminho {local_data_path}")

## 4. Definir a Entrada do Job de Treinamento

Agora que temos certeza de que o ativo de dados `diabetes-training:1` existe, podemos criar uma **Entrada (Input)** que o referencie. É esta entrada que passaremos para o nosso job de AutoML.

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# Cria uma referência (um "atalho") para o ativo de dados que acabamos de criar
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, 
    path="azureml:diabetes-training:1" # Sintaxe: azureml:<nome-do-ativo>:<versao>
)

## 5. Configurar o Job de Automated ML

Com os dados prontos, vamos configurar o job de AutoML.
Vamos definir:
* **A tarefa:** `classification` (classificação).
* **O cluster:** `aml-cluster` (o nome do cluster de computação que você criou).
* **Os dados de treino:** A entrada que definimos acima.
* **A coluna alvo:** `Diabetic` (o que queremos prever).
* **A métrica principal:** `AUC_weighted` (uma métrica robusta para classificação).
* **Limites:** Vamos limitar o job a 5 "trials" (modelos) para que ele termine rápido.

In [ ]:
from azure.ai.ml import automl

# Configura o job de classificação
classification_job = automl.classification(
    compute="aml-cluster", # <--- Verifique se este é o nome do seu cluster
    experiment_name="diabetes-classification-automl-project", # Um nome profissional para o seu experimento
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="AUC_weighted", # AUC é geralmente melhor que 'accuracy'
    n_cross_validations=5,
    enable_model_explainability=True
)

# Define os limites (para o job não rodar para sempre)
classification_job.set_limits(
    timeout_minutes=60, 
    max_trials=5, # Limita a 5 modelos diferentes
    enable_early_termination=True,
)

## 6. Submeter o Job e Monitorar

Finalmente, submetemos o job para o Azure ML. O `ml_client` nos dará um link para monitorar o progresso em tempo real no Azure ML Studio.

In [ ]:
# Submete o job para o Azure
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# Imprime o link do Studio para monitorar o job
aml_url = returned_job.studio_url
print(f"Job submetido! Monitore o progresso em: {aml_url}")

## 7. Próximos Passos

O job está em execução no Azure. Clique no link acima para ir ao Azure ML Studio.

Lá, você poderá ver:
1.  O job mudar de "Pendente" (Queued) para "Em Execução" (Running).
2.  Na aba **"Modelos + tarefas filhas"**, os modelos começarão a aparecer à medida que são treinados.
3.  Quando o job for **"Concluído" (Completed)**, você poderá selecionar o melhor modelo.
4.  Clique no melhor modelo e vá até a aba **"Explicações"** para ver o dashboard de explicabilidade do modelo (SHAP) e entender quais features (como Glicose, Idade, etc.) são mais importantes.